# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである．体系的にまとまってはいない．

## 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける．

関数内で配列を変更する場合には注意が必要である．以下に入力された配列を同じサイズの要素1の配列で置き換える，ということを目的として書かれた2つの関数がある．違いは`v`の後に`[:]`としているかどうかである．

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる．

In [2]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : [0.6280091741622842 0.9725327921727986; 0.5963608481273952 0.6912407844680865]
wrong : [0.6280091741622842 0.9725327921727986; 0.5963608481273952 0.6912407844680865]
right : [1.0 1.0; 1.0 1.0]


## Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると，1次元配列のリストを2次元配列に変換できた．Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる．

In [19]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Vector{Vector{Float64}}
Size : (5,)


In [20]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64, Matrix{Float64}}
Size : (5, 3)


以下は多次元配列の場合．`cat(...)`で配列を結合し，`permitedims`で転置する．

In [21]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Vector{Array{Float64, 3}}
Size : (6,)


In [22]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64, 4}
Size : (6, 3, 4, 5)


## 二項分布 (bernoulli distribution)のサンプリング

In [23]:
p = 0.7
N = 100

100

In [24]:
using BenchmarkTools

In [25]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 10000 samples with 54 evaluations.
 Range (min … max):  862.963 ns … 62.243 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     901.852 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.068 μs ±  1.164 μs  ┊ GC (mean ± σ):  3.38% ± 4.35%

  █▆▄▃▃▂▁          ▂▁                                          ▁
  ███████▇█▆▆▇▇▆▆▆▇███▇▇▆▆▆▆▅▅▅▅▅▃▅▄▅▅▃▄▅▄▃▃▂▃▂▃▄▄▂▄▂▃▄▃▄▃▃▂▂▂ █
  863 ns        Histogram: log(frequency) by time      3.33 μs <

 Memory estimate: 1.91 KiB, allocs estimate: 7.

In [26]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.67


In [27]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  970.000 ns … 147.660 μs  ┊ GC (min … max): 0.00% … 98.34%
 Time  (median):       1.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.345 μs ±   2.661 μs  ┊ GC (mean ± σ):  3.18% ±  1.71%

   █▄▁                                                           
  ▄███▇▄▃▂▂▃▃▃▃▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂ ▃
  970 ns           Histogram: frequency by time         3.35 μs <

 Memory estimate: 1.53 KiB, allocs estimate: 6.

In [28]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.66


## "splat" operator ... 
https://docs.julialang.org/en/v1/base/base/#...

## broadcastingの回避

In [34]:
foo(a,b) = sum(a) + b

foo (generic function with 1 method)

In [35]:
println(foo.(Ref([1,2]),[3,4,5]))
println(foo.(([1,2],), [3,4,5]))
println(foo.([[1,2]], [3,4,5]))

[6, 7, 8]
[6, 7, 8]
[6, 7, 8]


## `var`の使用

In [36]:
var"log(θ+μ)" = 10

10